In [37]:
from eodhd import APIClient
import os
import urllib, json
import pandas as pd 
import numpy as np 
import datetime
import yfinance as yf






In [38]:
#EOD API 
YOUR_API_KEY="647950fbb96692.96123493"
api = APIClient(YOUR_API_KEY)


In [50]:
def read_url(url=" "):
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    df=pd.DataFrame.from_dict(data)
    return df

In [57]:
dfExchange=read_url(url=f"https://eodhistoricaldata.com/api/exchanges-list/?api_token={YOUR_API_KEY}")
                    

In [70]:
exchangeList = dfExchange["Code"].tolist()
dfGlobalTicker=[]
for exchangeCode in exchangeList[:2]:
    dfTicker = read_url(url=f"https://eodhistoricaldata.com/api/exchange-symbol-list/{exchangeCode}?api_token={YOUR_API_KEY}&fmt=json")
    dfGlobalTicker.append(dfTicker)

In [88]:
dfGlobalTicker=pd.concat(dfGlobalTicker,axis=0, ignore_index=True)

In [92]:
tickerTypes=dfGlobalTicker["Type"].unique().tolist()

In [95]:
dfStockTickers = dfGlobalTicker[dfGlobalTicker["Type"]=="Common Stock"]

In [301]:
def EOD_Data(exchange=[],tickerList=[]):
    
    allDataKeys=['General', 'Highlights', 'Valuation', 'SharesStats', 'Technicals',
           'SplitsDividends', 'AnalystRatings', 'Holders', 'InsiderTransactions',
           'ESGScores', 'outstandingShares', 'Earnings', 'Financials']

    #Key Sub Classification
    #ESGScource - Outdated 
    companyInfoDataKeys = ['General', 'Highlights', 'Valuation', 'SharesStats', 'Technicals', 'AnalystRatings']
    splitsDividendKey = ["SplitsDividend"]
    shareholdersKey = ["Holders"]
    iTransactionKey = ["InsiderTransactions"]
    osSharesKey = ['outstandingShares']
    earningsKey = ["Earnings"]
    financialsKey = ["Financials"]


    companyInfo = [] 
    officersInfo = []
    listingsInfo = []
    shareholderInfo = []
    shareOutstandInfo = []
    insiderTransacInfo = []
    earningHist = []
    earningTrend  = []
    earningAnnual = [] 
    statements = ['Balance_Sheet','Cash_Flow','Income_Statement']
    periods = ["quarterly","yearly"]
    annualFS=[]
    quarterFS=[]
    noData = {}
    noDataDf = []
    count = 1



    for ticker in tickerList:
        url=f"https://eodhistoricaldata.com/api/fundamentals/{ticker}.{exchange}?api_token={YOUR_API_KEY}"
        response = urllib.request.urlopen(url)
        data = json.loads(response.read())


        #Company Info
        dfCI = []
        for key in companyInfoDataKeys:
            dfCI.append(pd.DataFrame.from_dict([data[key]]))

        dfCompanyInfo=pd.concat(dfCI,axis=1)
        companyInfo.append(dfCompanyInfo)


        #Company Financials 
        dfF= []
        for period in periods:
            for statement in statements:
                dfF.append(pd.DataFrame.from_dict(data['Financials'][statement][period]))

            dfFS=pd.concat(dfF,axis=1,ignore_index=True).transpose()
            dfFS["Ticker"]=ticker
            if period == "quarterly":
                  quarterFS.append(dfFS)
                  dfF = []
            else:
                  annualFS.append(dfFS)
                  dfF = []
    return companyInfo,annualFS,quarterFS

In [294]:
def convert_to_space_upper(text):
    import re
    if re.match(r'^[A-Za-z0-9_]+$', text):
        if re.match(r'^[a-z]+(_[a-z]+)*$', text):
            return text.replace('_', ' ').upper()
        else:
            return re.sub(r'([a-z0-9])([A-Z])', r'\1 \2', text).upper()
    else:
        return text.upper()
    
def concatList(clist=[]):
    df=pd.concat(clist,ignore_index=True)
    df.columns = df.columns.map(convert_to_space_upper)
    return df

In [306]:
companyInfo,annualFS,quarterFS = EOD_Data(exchange="US",tickerList=["AAPL","MSFT"])

In [307]:
dfF=concatList(annualFS)
dfQ=concatList(quarterFS)
dfC=concatList(companyInfo)

In [ ]:
df.rename(columns={"CODE": "TICKER", "COUNTRY NAME": "COUNTRY"})

In [310]:
dfM = pd.read_csv(r"D:\EQUITY DATA\Site Data-Others\MetricRef.csv")

In [362]:
#Changing Quarterly Financials to USD
def QFUSD(dFF=[],dfM=[]):
    import yfinance as yf
    dFF.columns = dFF.columns.str.lstrip()
    dFF["DATE"]=pd.to_datetime(dFF["DATE"],errors="coerce")
   
    dFF["YEAR"]=dFF["DATE"].dt.year
    dFF["QUARTER"]= dFF["DATE"].dt.quarter
    dFF["MONTH"] = dFF["DATE"].dt.month
 
    sd="1980-1-1"
    ed = datetime.date.today()


    currencyList=dFF["CURRENCY SYMBOL"].unique().tolist()
    currencyList.remove("USD")
    #currencyList.remove(np.nan)
    
    currencies=currencyList
    currencyTick = []
    for currency in currencies:
        currencyTick.append(currency+"USD=X")

    currencyData = yf.download(currencyTick,start=sd,end=ed)
    exRates=currencyData["Close"]
    exQ = exRates.resample("Q").mean().reset_index()
    exQ["YEAR"] = exQ["Date"].dt.year
    exQ["QUARTER"] = exQ["Date"].dt.quarter
    exM =  exRates.resample("M").mean().reset_index()
    exM["YEAR"] = exM["Date"].dt.year
    
    exM["MONTH"] = exM["Date"].dt.month

    dFFex=dFF[dFF["CURRENCY SYMBOL"].isin(currencies)][["TICKER","DATE","YEAR","MONTH","QUARTER","CURRENCY SYMBOL"]]

    zlist = []
    for currency in currencies:
        cCode=currency+"USD=X"

        x=dFFex[dFFex["CURRENCY SYMBOL"]==currency]

        yFp = exQ[[cCode,"YEAR","QUARTER"]]
        yFp.columns = ["ExRate_forperiod","YEARfp","QUARTERfp"]


        yAd = exM[[cCode,"YEAR","MONTH"]]
        yAd.columns=["ExRate_asonDate","YEARas","MONTHas"]

        z=pd.merge(x,yFp,left_on=["YEAR","QUARTER"],right_on=["YEARfp","QUARTERfp"])
        zqq=pd.merge(z,yAd,left_on=["YEAR","MONTH"],right_on=["YEARas","MONTHas"])

        zlist.append(zqq)

    erT=pd.concat(zlist)


    erT.drop(["YEAR","MONTH","QUARTER","CURRENCY SYMBOL","YEARfp","QUARTERfp","YEARas","MONTHas"],axis=1,inplace=True)

    dFFex=pd.merge(dFF,erT,left_on=["TICKER","DATE"],right_on=["TICKER","DATE"])

    for met in dfM[dfM["Statement"]=="BS"]["title"]:
                dFFex[met]=dFFex[met]*dFFex["ExRate_asonDate"]

    for met in dfM[dfM["Statement"].isin(["CF","IS"])]["title"]:
                dFFex[met]=dFFex[met]*dFFex["ExRate_forperiod"]

    usdDF=dFF[~dFF["CURRENCY SYMBOL"].isin(currencies)]

    usdF=pd.concat([dFFex,usdDF])

    usdF.loc[usdF["CURRENCY SYMBOL"]=="USD","ExRate_asonDate"]=1
    usdF.loc[usdF["CURRENCY SYMBOL"]=="USD","ExRate_forperiod"]=1
    
    usdF.loc[usdF["CURRENCY SYMBOL"].isin([np.nan]),"ExRate_asonDate"]=1
    usdF.loc[usdF["CURRENCY SYMBOL"].isin([np.nan]),"ExRate_forperiod"]=1
    
    usdF.dropna(subset=["ExRate_forperiod"],inplace=True)
    
    return usdF

In [363]:
QFUSD(dFF=dfQ,dfM=dfM)

[*********************100%***********************]  0 of 0 completed


ValueError: No objects to concatenate

In [338]:
dfC["TICKER"]=dfC["CODE"]

In [344]:
#VARIABLES

sector = 'SECTOR'
industry = "INDUSTRY"
ticker = "TICKER"
coName = "NAME"
year = "YEAR"
marketCap='MARKET CAPITALIZATION'
updatedTicker="YF TICKER"
country = "COUNTRY NAME"
Date ="DATE"
roic = 'ROIC'
revenue = 'TOTAL REVENUE'
rev_type = "TOTAL REVENUE"
fcf = 'FREE CASH FLOW'
gm = 'GROSS PROFIT MARGIN'
ebitda_m = 'EBITDA MARGIN'
npm = 'NET PROFIT MARGIN'
nprofit = "NET INCOME"
gp = "GROSS PROFIT"
ebitda = "EBITDA"
assets = "TOTAL ASSETS"
liab="TOTAL LIAB"
equity="TOTAL STOCKHOLDER EQUITY"
cfo = "TOTAL CASH FROM OPERATING ACTIVITIES"
cfi = "TOTAL CASHFLOWS FROM INVESTING ACTIVITIES"
cff = "TOTAL CASH FROM FINANCING ACTIVITIES"
d_e = 'DEBT TO EQUITY'
c_r = 'CURRENT RATIO'
pe = 'PRICE TO EARNINGS RATIO (TTM)'
pcf = 'PRICE TO FREE CASH FLOW (TTM)'
prev = "PRICE TO REVENUE RATIO (TTM)"
st1 = "IS"
st2 = "CF"
st3 = "Ratio"
st4 = "Ratio"
IS = "IS"
BS = "BS"
CF = "CF"
OT = "Ratio "
mScale="MARKET CAP SCALE"
indexUS=["S&P500","NASDAQ100","DOW30"]
indexIND = ["SECTORIAL INDEX","MARKET CAP INDEX"]

In [345]:
dfC

,CODE,TYPE,NAME,EXCHANGE,CURRENCY CODE,CURRENCY NAME,CURRENCY SYMBOL,COUNTRY NAME,COUNTRY ISO,OPEN FIGI,...,SHORT RATIO,SHORT PERCENT,RATING,TARGET PRICE,STRONG BUY,BUY,HOLD,SELL,STRONG SELL,TICKER
0,AAPL,Common Stock,Apple Inc,NASDAQ,USD,US Dollar,$,USA,US,BBG000B9XRY4,...,2.00,0.0070,4.2500,178.88,24,10,8,1,1,AAPL
1,MSFT,Common Stock,Microsoft Corporation,NASDAQ,USD,US Dollar,$,USA,US,BBG000BPH459,...,1.51,0.0057,4.4118,331.45,30,14,6,0,1,MSFT


In [346]:
dfCsel=dfC[[ticker,coName,marketCap,sector,industry,country]]

In [348]:
dfF = pd.merge(dfF,dfCsel,left_on=ticker,right_on=ticker)

In [349]:
dfF

,DATE,FILING DATE,CURRENCY SYMBOL,TOTAL ASSETS,INTANGIBLE ASSETS,EARNING ASSETS,OTHER CURRENT ASSETS,TOTAL LIAB,TOTAL STOCKHOLDER EQUITY,DEFERRED LONG TERM LIAB,...,DISCONTINUED OPERATIONS,NET INCOME FROM CONTINUING OPS,NET INCOME APPLICABLE TO COMMON SHARES,PREFERRED STOCK AND OTHER ADJUSTMENTS,TICKER,NAME,MARKET CAPITALIZATION,SECTOR,INDUSTRY,COUNTRY NAME
0,2022-09-30,2022-10-27,USD,352755000000.00,None,None,21223000000.00,302083000000.00,50672000000.00,None,...,NaN,NaN,NaN,NaN,AAPL,Apple Inc,2846108418048,Technology,Consumer Electronics,USA
1,2021-09-30,2021-10-29,USD,351002000000.00,None,None,14111000000.00,287912000000.00,63090000000.00,None,...,NaN,NaN,NaN,NaN,AAPL,Apple Inc,2846108418048,Technology,Consumer Electronics,USA
2,2020-09-30,2020-10-30,USD,323888000000.00,None,None,11264000000.00,258549000000.00,65339000000.00,None,...,NaN,NaN,NaN,NaN,AAPL,Apple Inc,2846108418048,Technology,Consumer Electronics,USA
3,2019-09-30,2019-10-31,USD,338516000000.00,None,None,12352000000.00,248028000000.00,90488000000.00,None,...,NaN,NaN,NaN,NaN,AAPL,Apple Inc,2846108418048,Technology,Consumer Electronics,USA
4,2018-09-30,2018-11-05,USD,365725000000.00,None,None,12087000000.00,258578000000.00,107147000000.00,2797000000.00,...,NaN,NaN,NaN,NaN,AAPL,Apple Inc,2846108418048,Technology,Consumer Electronics,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,1990-06-30,1990-06-30,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,MSFT,Microsoft Corporation,2493863165952,Technology,Software-Infrastructure,USA
214,1989-06-30,1989-06-30,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,MSFT,Microsoft Corporation,2493863165952,Technology,Software-Infrastructure,USA
215,1988-06-30,1988-06-30,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,MSFT,Microsoft Corporation,2493863165952,Technology,Software-Infrastructure,USA
216,1987-06-30,1987-06-30,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,MSFT,Microsoft Corporation,2493863165952,Technology,Software-Infrastructure,USA
